In [1]:
import torch as t

def tick(alpha, vx, vy):
    lx = t.cos(alpha)
    ly = t.sin(alpha)
    speed_proj = vx
    vy += 0.08 * (-1.0 + lx * lx * 0.75)
    if vy < 0 and lx > 0:
        d = vy * -0.1 * lx * lx
        vx += d
        vy += d
    if ly > 0 and lx > 0:
        d = speed_proj * ly * 0.04
        vx -= d
        vy += d * 3.2
    if lx > 0:
        vx += (speed_proj - vx) * 0.1
    return vx * 0.9900000095367432, vy * 0.9800000190734863




In [36]:

import torch.nn as nn

# Defining input size, hidden layer size, output size and batch size respectively
n1_in, n1_h1, n1_h2 , n1_out, batch_size = 3, 10, 10, 1, 10

actor = nn.Sequential(nn.Linear(n1_in, n1_h1),
   nn.ReLU(),
   nn.Linear(n1_h1, n1_h2),
   nn.ReLU(),
   nn.Linear(n1_h1, n1_h2),
   nn.ReLU(),
   nn.Linear(n1_h1, n1_h2),
   nn.ReLU(),
   nn.Linear(n1_h2, n1_out))

n2_in, n2_h1, n2_h2 , n2_out = 4, 10, 10, 1
critic = nn.Sequential(nn.Linear(n2_in, n2_h1),
   nn.ReLU(),
   nn.Linear(n2_h1, n2_h2),
   nn.ReLU(),
   nn.Linear(n2_h1, n2_h2),
   nn.ReLU(),
   nn.Linear(n2_h1, n2_h2),
   nn.ReLU(),
   nn.Linear(n2_h2, n2_out))

class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.actor = actor
        self.critic = critic

    def forward(self, x):
        output = self.actor(x)
        output = self.critic(t.cat((output,x),1))
        return output


class MyLoss(nn.Module):
    # 不要忘记继承Module
    def __init__(self):
        super(MyLoss, self).__init__()

    def forward(self, output):
        # 不要忘记返回scalar
        return t.mean(-output)
net = model()


# Construct the loss function
criterion1 = nn.MSELoss()
criterion2 = MyLoss()
# Construct the optimizer (Stochastic Gradient Descent in this case)
optimizer1 = t.optim.SGD(critic.parameters(), lr = 0.01, weight_decay=0.01)
optimizer2 = t.optim.SGD(net.parameters(), lr = 0.01)




In [3]:
t.cuda.is_available()


False

In [43]:
R = []
for i_episode in range(2000):
    sa = []
    r = []
    Hlist = []
    # H = t.rand(1)*200+10
    H = t.tensor([300.0])
    H_ = (H).tolist()[0]
    IniInputlist = []
    X = 0
    lastX = 0
    vx = t.tensor([0.0])
    vy = t.tensor([0.0])
    while H>0:
        IniInpu = t.cat((H,vx,vy),0)
        action = actor(IniInpu)
        IniInputlist.append(IniInpu.tolist())
        alpha = action
        print(alpha,end="")
        
        if alpha>3.1415/2 or alpha<-3.1415/2 or t.isnan(alpha[0]):
            alpha = t.tensor([0.0])
        vx, vy = tick(alpha,vx,vy)
        # print(H,alpha,vx,vy,end="")
        # print(H)
        H = H + vy
        X = X + vx
        
        action = alpha.tolist()
        action= action + IniInpu.tolist()
        # print(action)
        sa.append(action)
        r.append([lastX])
        lastX = X
    print("行进",vx,"高度",H,end="")
    for oner in r:
        oner[0] = X - oner[0]
    R.append(X/H_)
    sa = t.tensor(sa)
    r = t.tensor(r)
    print(sa.shape,r.shape)
    # Gradient Descent
    for epoch in range(20):

        for i in range(5):
            
            y_pred = critic(sa)
            loss1 = criterion1(y_pred,r)
            optimizer1.zero_grad()
            loss1.backward()
            optimizer1.step()
        
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = net(t.tensor(IniInputlist))

        # Compute and print loss
        loss2 = criterion2(y_pred)
        if epoch%5==0:
            print('i_episode: ',i_episode, 'epoch: ', epoch,' loss1: ', loss1.tolist(),' loss2: ', loss2.tolist())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer2.zero_grad()

        # perform a backward pass (backpropagation)
        loss2.backward()

        # Update the parameters
        optimizer2.step()

import matplotlib.pyplot as plt

plt.plot(R)

d_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad_fn=<AddBackward0>)tensor([nan], grad

KeyboardInterrupt: 